# ランキング学習（LGBMRanker）による勝率分析

## 本題
true skillsを用いて各馬の強さを数値化したい
v02とは違い、オッズを特徴量から除外してどう結果が変わるのかを調べる

# 結果
predict\LGBMRanker02.5\test_refund.ipynbの出力結果に出てるように著しく性能が下がる  
オッズは重要な特徴量とみなしてよいことが分かったが、オッズを含めたうえで回収率を向上させるための案が必要。  
次案はレースのクラスタリング、買い方の最適化、調教データの取り込み

- 参考：https://qiita.com/kazuktym/items/3aa1f9e10d9231cf5cab#%E3%83%AC%E3%83%BC%E3%82%B9%E3%81%AF%E7%9B%B8%E5%AF%BE%E8%A9%95%E4%BE%A1%E3%81%A7%E4%BA%88%E6%83%B3%E3%81%97%E3%81%9F%E3%81%84

In [1]:
import pandas as pd
import tqdm

import os
import sys
sys.path.append('../../')
import common.com as common
from common import const,utils
import copy
from datetime import datetime
import scraping.scraping as scraping


In [2]:
# # DBにある全てのレースデータを重複を排除して取得（2024/06/01実行時点では2013-01_2023-12のデータ）
_data = common.get_race_data_distinct()
data=pd.DataFrame(_data,columns=const.ORIGIN_RACE_DATA_HEADERS)

In [3]:
# data=pd.read_csv('D://work//Programming//data//race_datas//2013-01_2023-12.csv',header=0)

In [4]:
# ラベルエンコーディング(LabelEncoder)
from sklearn.preprocessing import LabelEncoder

encode_tar_cols = [
'性',
'斤量',
'芝かダートか',
'回り',
'馬場状態',
'天気',]

encoded = copy.deepcopy(data)

le = LabelEncoder()
for col in encode_tar_cols:
    encoded[col] = le.fit_transform(encoded[col].values)

In [5]:
# 着順関連度で重み付け
ORDER_RELATED = {1:30,2:28,3:26,4:24,5:22}
def get_order_related(order):
    try:
        return ORDER_RELATED[order]
    except KeyError:
        return 0
    
encoded['着順関連度'] = encoded['着順'].map(get_order_related)

In [6]:
encoded['オッズ'] = encoded['オッズ'].replace('---',0).astype(float)

In [1]:

# objectになってしまうので、floatに変換
if encoded[col].dtype == 'object':
    if col in ['レースID','馬番','枠番','齢','馬のID','騎手のID','距離','競馬場ID']:
        encoded[col] = encoded[col].astype('int64')
    else:
        encoded[col] = encoded[col].astype(float)

In [8]:
# レースIDでグループ化
grouped_data = encoded.groupby('レースID')

In [9]:
import trueskill
mu = 25.
sigma = mu / 3.
beta = sigma / 2.
tau = sigma / 100.
draw_probability = 0.001
backend = None
env = trueskill.TrueSkill(
    mu=mu, sigma=sigma, beta=beta, tau=tau,
    draw_probability=draw_probability, backend=backend)


In [10]:
result=pd.read_csv(const.BASE_DIR+'predict//LGBMRanker02.5//ranked_race_data.csv')

In [11]:
# 8:2の割合で、学習用と評価用に分割
grouped_data_list = list(result.groupby('レースID'))
learn_count = round(len(grouped_data_list) * 0.8)

learn_grouped_data = grouped_data_list[:learn_count]
test_grouped_data = grouped_data_list[learn_count:]

# 8:2の割合で、訓練用と検証用に分割
train_count = round(len(learn_grouped_data) * 0.8)
train_grouped_data = learn_grouped_data[:train_count]
valid_grouped_data = learn_grouped_data[train_count:]

In [12]:
# 直前のレースまでのレートを特徴量に含めたうえで学習
target_variables=copy.deepcopy(const.VARIABLE)
target_variables.remove('オッズ')
target_variables.append('直前レート')
X_train = pd.concat([item[1].loc[:,target_variables] for item in train_grouped_data])
X_valid = pd.concat([item[1].loc[:,target_variables] for item in valid_grouped_data])
X_test = pd.concat([item[1].loc[:,target_variables] for item in test_grouped_data])
y_train = pd.concat([item[1].loc[:,'着順関連度'] for item in train_grouped_data])
y_valid = pd.concat([item[1].loc[:,'着順関連度'] for item in valid_grouped_data])
y_test = pd.concat([item[1].loc[:,'着順関連度'] for item in test_grouped_data])

In [13]:
# クエリーデータ(レース単位のデータ数のリスト)を作成
train_query = X_train.groupby('レースID').size().values.tolist()
valid_query = X_valid.groupby('レースID').size().values.tolist()

In [14]:
import lightgbm as lgb

model = lgb.LGBMRanker(
  random_state=100,              # 乱数シード
  n_estimators=500,              # 決定木の個数(default:100)
  learning_rate=0.01,            # 学習率(default:0.1)
  num_leaves=100,                 # 決定木にある分岐の個数(default:31)
  max_depth=-1,                  # 決定木の深さの最大値(default:-1)
  min_child_samples=150,         # 一つの葉に含まれる最小データ数(default:20) 
)

model.fit(X_train, y_train,
  group=train_query,             # 訓練用クエリーデータ
  eval_set=[(X_valid, y_valid)], # 学習時に用いる検証用データ
  eval_group=[valid_query],      # 検証用クエリーデータ
  eval_metric='ndcg',            # 学習時の評価手法
  eval_at=[1, 2, 3,4,5]              # 学習時の評価対象順位
)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014879 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1352
[LightGBM] [Info] Number of data points in the train set: 347498, number of used features: 19


LGBMRanker(learning_rate=0.01, min_child_samples=150, n_estimators=500,
           num_leaves=100, random_state=100)

In [15]:
y_pred = model.predict(X_test, num_iteration=model.best_iteration_)

# 予測結果や関連度をDataFrameに連結
df_pred = pd.DataFrame({
    'レースID': X_test['レースID'],
    '馬番号': X_test['馬のID'],
    '予想スコア':  y_pred,
    '着順関連度': y_test,
})

In [16]:
df_pred.to_csv(const.BASE_DIR+'predict//LGBMRanker02//predict.csv')

In [17]:
# 特徴量重要度の抽出
df_importances = pd.DataFrame({'columns':X_train.columns, 'importances':model.feature_importances_})
df_importances.sort_values('importances', ascending=False, inplace=True)
print(df_importances)

   columns  importances
2       馬番        14197
1       枠番        10972
16   競馬場ID         4396
11  芝かダートか         3875
12      距離         3827
4        齢         2909
17   着順関連度         2500
8       人気         1834
0    レースID         1535
18   直前レート         1003
6     馬のID          821
13      回り          443
9       体重          306
5       斤量          299
7    騎手のID          294
3        性          159
10    体重変化           93
15      天気           27
14    馬場状態           10


In [18]:
from sklearn.metrics import ndcg_score
# クエリーごとにNDCGを計算し、その平均値を算出
ndcg_score = df_pred.groupby('レースID').apply(lambda d: ndcg_score([d['着順関連度']], [d['予想スコア']], k=5)).mean()
print(ndcg_score)

1.0


C:\Users\ionde\AppData\Local\Temp\ipykernel_24768\245059073.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ndcg_score = df_pred.groupby('レースID').apply(lambda d: ndcg_score([d['着順関連度']], [d['予想スコア']], k=5)).mean()


In [19]:
import pickle
# モデルの保存
file_dir = const.BASE_DIR+'predict//LGBMRanker02.5//LGBMRanker02.5.pickle'
pickle.dump(model, open(file_dir, 'wb'))